In [1]:
#最低限必要な情報:物件名・住所・間取り・家賃・階数
#検索エリア：23区、2K以上、家賃管理費込み22万円未満の物件情報

In [2]:
# ライブラリのインポート
from bs4 import BeautifulSoup
import requests
import csv
import time

# 物件情報を抽出する関数
def scrape_property_data(soup):
    property_data = []
    property_panels = soup.find_all("div", class_="PropertyPanel_propertyPanel__MqCpF")
    for panel in property_panels:
        title = panel.find("div", class_="PropertyPanelBuilding_buildingTitle__NbWmb").text.strip().split("】")[1].strip()  # タイトルの抽出
        address = panel.find("p", class_="is-mt5").text.strip()  # 住所の抽出
        access_info = panel.find_all("div", class_="PropertyPanelBuilding_buildingInformationSection__AMRsh")[0]
        access = "、".join([p.text for p in access_info.find_all("p") if not p.get("class")])  # アクセス情報の抽出
        age = panel.find_all("div", class_="PropertyPanelBuilding_buildingInformationSection__AMRsh")[1].find("p").text.strip()  # 築年数の抽出
        floor = panel.find_all("div", class_="PropertyPanelBuilding_buildingInformationSection__AMRsh")[1].find_all("p")[1].text.strip()  # 建物の階数の抽出
        fee = panel.find("div", class_="PropertyPanelRoom_rentPrice__HO4Jp").text.split("(")[0].strip()  # 賃料の抽出
        management_fee = panel.find("div", class_="PropertyPanelRoom_rentPrice__HO4Jp").text.split("(")[1].replace(")", "").strip() if len(panel.find("div", class_="PropertyPanelRoom_rentPrice__HO4Jp").text.split("(")) > 1 else ""  # 管理費の抽出
        if "管理費" in management_fee:
            management_fee = management_fee.replace("管理費", "").strip()
        
        # 敷金と礼金の抽出
        initial_prices = panel.find("div", class_="PropertyPanelRoom_initialPrices__naYEA")
        if initial_prices:
            price_elements = initial_prices.find_all("li")
            deposit = price_elements[0].text.strip() if price_elements else ""
            gratuity = price_elements[1].text.strip() if len(price_elements) > 1 else ""
        else:
            deposit, gratuity = "", ""
        
        room_info_element = panel.find("span", class_="is-ml5")
        if room_info_element:
            room_info_text = room_info_element.get_text().split("/")
            floor_plan = room_info_text[1].strip() if len(room_info_text) > 1 else ""  # 間取りの抽出
            area = room_info_text[2].strip() if len(room_info_text) > 2 else ""  # 面積の抽出
        else:
            floor_plan, area = "", ""

        # 仲介手数料の抽出
        RoomItem = panel.select("a.PropertyPanelRoom_roomItem__3bVhC")[0].select("div.PropertyPanelRoom_roomItemRightContent__UzfRX")[0].select("div")[2]
        if "PropertyPanelRoom_isRed__HXtHO" in str(RoomItem):
            charge = RoomItem.select("span.PropertyPanelRoom_isRed__HXtHO")[0].text
            charge = "0円" if "無料" in charge else charge
        else:
            charge = RoomItem.select("span")[1].text

        property_data.append([title, address, access, age, floor, fee, management_fee, deposit, gratuity, floor_plan, area, charge])
    return property_data

def main():
    base_url = "https://airdoor.jp/list?si=d-131016-131024-131032-131041-131059-131067-131075-131083-131091-131105-131113-131121-131130-131148-131156-131164-131172-131181-131199-131202-131211-131229-131237&ur=220000&iaf=1&fp=d-2_k-2_dk-2_ldk-3_k-3_dk-3_ldk-4_more&p={}"
    data_list = []

    page = 1
    while True:
        print(f"ページ {page} を取得中...")
        target_url = base_url.format(page)
        r = requests.get(target_url)

        if r.status_code != 200:
            print("最後のページまで取得しました。")
            break

        soup = BeautifulSoup(r.text, 'html.parser')
        property_data = scrape_property_data(soup)

        # 物件情報がない場合、ループを終了
        if not property_data:
            print(f"ページ {page} に物件情報がありません。スクレイピングを終了します。")
            break

        data_list.extend(property_data)
        page += 1
        time.sleep(1)  # 1秒間のウェイト

    # CSVファイルへの書き込み
    with open('airdoor_data.csv', 'w', newline='', encoding='utf-8-sig') as file:
        writer = csv.writer(file)
        writer.writerow(['Title', 'Address', 'Access', 'Age', 'Floor', 'Fee', 'Management Fee', 'Deposit', 'Gratuity', 'Floor Plan', 'Area', 'Charge'])
        writer.writerows(data_list)

    print("スクレイピングが完了しました。")

if __name__ == "__main__":
    main()


ページ 1 を取得中...
ページ 2 を取得中...
ページ 3 を取得中...
ページ 4 を取得中...
ページ 5 を取得中...
ページ 6 を取得中...
ページ 7 を取得中...
ページ 8 を取得中...
ページ 9 を取得中...
ページ 10 を取得中...
ページ 11 を取得中...
ページ 12 を取得中...
ページ 13 を取得中...
ページ 14 を取得中...
ページ 15 を取得中...
ページ 16 を取得中...
ページ 17 を取得中...
ページ 18 を取得中...
ページ 19 を取得中...
ページ 20 を取得中...
ページ 21 を取得中...
ページ 22 を取得中...
ページ 23 を取得中...
ページ 24 を取得中...
ページ 25 を取得中...
ページ 26 を取得中...
ページ 27 を取得中...
ページ 28 を取得中...
ページ 29 を取得中...
ページ 30 を取得中...
ページ 31 を取得中...
ページ 32 を取得中...
ページ 33 を取得中...
ページ 34 を取得中...
ページ 35 を取得中...
ページ 36 を取得中...
ページ 36 に物件情報がありません。スクレイピングを終了します。
スクレイピングが完了しました。


In [10]:
import pandas as pd
import re
import csv

# CSV ファイルを読み込む
df = pd.read_csv('airdoor_data.csv')

# Access の分割
def split_access(access):
    # 分割用の正規表現
    pattern = r'(\D+?) (\D+?)駅 徒歩(\d+)分'
    matches = re.findall(pattern, access)
    if len(matches) == 1:
        return [*matches[0], '', '', '']
    elif len(matches) == 2:
        return [item for sublist in matches for item in sublist]
    else:
        return ['', '', '', '', '', '']

# Access 列を分割
access_cols = ['access1_route', 'access1_nearest_station', 'access1_walking_minutes',
               'access2_route', 'access2_nearest_station', 'access2_walking_minutes']
df[access_cols] = df['Access'].apply(lambda x: pd.Series(split_access(str(x))))

# access2_route 列からカンマを削除
df['access2_route'] = df['access2_route'].str.lstrip('、')

# Age 列の変換
def convert_age(value):
    if '新築' in value:
        return 0
    else:
        match = re.search(r'築(\d+)年', value)
        return int(match.group(1)) if match else 0

df['Age'] = df['Age'].apply(convert_age)

# Floor 列の変換
df['Floor'] = df['Floor'].str.extract(r'(\d+)階建').astype(int)

# 金額関連の列の変換
def convert_fee(value):
    if value == '無料':
        return 0
    else:
        value = value.replace('円', '').replace(',', '')
        if '万' in value:
            value = value.replace('万', '')
            return float(value) * 10000
        else:
            return float(value)

# Fee, Management Fee, Charge 列の変換
for col in ['Fee', 'Management Fee', 'Charge']:
    df[col] = df[col].apply(convert_fee)

# Deposit, Gratuity 列の変換
for col in ['Deposit', 'Gratuity']:
    df[col] = df[col].apply(convert_fee)

# Area 列の変換
df['Area'] = df['Area'].str.replace('㎡', '').astype(float)

# ヘッダーを指定
headers = ['title', 'address', 'access', 'age', 'floor', 'fee', 'management_fee', 'deposit', 'gratuity', 'floor_plan', 'area', 'charge',
           'access1_route', 'access1_nearest_station', 'access1_walking_minutes', 'access2_route', 'access2_nearest_station', 'access2_walking_minutes']

# 変換されたデータを CSV ファイルに保存
data_list = df.values.tolist()

with open('cleaned_airdoor_data.csv', 'w', newline='', encoding='utf-8-sig') as file:
    writer = csv.writer(file)
    writer.writerow(headers)
    writer.writerows(data_list)

C:\Users\udano\AppData\Local\Temp\ipykernel_1980\4042364468.py:23: FutureWarning: Returning a DataFrame from Series.apply when the supplied function returns a Series is deprecated and will be removed in a future version.
  df[access_cols] = df['Access'].apply(lambda x: pd.Series(split_access(str(x))))


In [18]:
import pandas as pd

# クレンジングされたデータを読み込む
df = pd.read_csv('cleaned_airdoor_data.csv')

# 指定された列を整数に変換
columns_to_convert = ['fee', 'management_fee', 'deposit', 'gratuity', 'charge', 'access1_walking_minutes', 'access2_walking_minutes']
df[columns_to_convert] = df[columns_to_convert].fillna(0).astype(int)

# 重複するデータの件数を取得
duplicates_count = df.duplicated(subset=['title', 'floor', 'fee', 'floor_plan', 'area']).sum()
print(f"重複するデータの件数: {duplicates_count}")

# 重複するデータを表示
duplicates = df[df.duplicated(subset=['title', 'floor', 'fee', 'floor_plan', 'area'], keep=False)]
print("重複するデータ:")
print(duplicates)

# 重複を削除
df.drop_duplicates(subset=['title', 'floor', 'fee', 'floor_plan', 'area'], inplace=True)

# 重複が削除された後のデータフレームを表示（オプション）
print("重複削除後のデータ:")
print(df.head())

重複するデータの件数: 0
重複するデータ:
Empty DataFrame
Columns: [title, address, access, age, floor, fee, management_fee, deposit, gratuity, floor_plan, area, charge, access1_route, access1_nearest_station, access1_walking_minutes, access2_route, access2_nearest_station, access2_walking_minutes]
Index: []
重複削除後のデータ:
           title         address                                  access  age  \
0    ベルファース駒沢三丁目    東京都世田谷区駒沢３丁目  東急田園都市線 駒沢大学駅 徒歩10分、東急田園都市線 桜新町駅 徒歩13分    0   
1  Ｋｏｌｅｔ堀切菖蒲園＃０５    東京都葛飾区東堀切２丁目      京成本線 お花茶屋駅 徒歩12分、京成本線 堀切菖蒲園駅 徒歩14分    1   
2   市ヶ谷加賀町アパートA棟  東京都新宿区市谷加賀町２丁目                       都営大江戸線 牛込柳町駅 徒歩5分   54   
3       Ｋｏｌｅｔ扇大橋      東京都足立区扇１丁目                    日暮里舎人ライナー 扇大橋駅 徒歩11分    0   
4     グランカーサ三ノ輪Ⅱ    東京都台東区日本堤２丁目                     東京メトロ日比谷線 三ノ輪駅 徒歩5分    0   

   floor     fee  management_fee  deposit  gratuity floor_plan   area  charge  \
0      3  119000           10000   119000         0         2K  25.28       0   
1      3  188000               0   188000        

In [19]:
import gspread
from google.oauth2 import service_account

# GoogleSheetsAPI、GoogleDriveAPI、及び認証鍵の指定
SCOPES = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive']

# サービスアカウントキーファイルのパスを指定
SERVICE_ACCOUNT_FILE = "C:\\Users\\udano\\OneDrive\\デスクトップ\\Tech0\\Step3-1\\SUUMO\\googlespreadsheet.json"

# Service Accountの認証情報を取得
credentials = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)

# 認証情報を用いてGoogleSheetsにアクセス
gc = gspread.service_account(filename=SERVICE_ACCOUNT_FILE)

# 対象のスプレッドシートとワークシートを指定
SPREADSHEET_KEY = "1-l-E6gb82lZQKlDSMpQX1LhH_z8bLG-u591WAzJv8p4"
worksheet = gc.open_by_key(SPREADSHEET_KEY).worksheet("シート2")

# データをスプレッドシートに書き込む前に、浮動小数点数値を文字列に変換
duplicates = duplicates.astype(str)

# データフレームを 2 次元リストに変換
data_values = df.astype(str).values.tolist()

# スプレッドシートの既存のデータをクリア
worksheet.clear()

# ヘッダー行のデータ（列名）
header = df.columns.tolist()

# ヘッダーをスプレッドシートの1行目に挿入
worksheet.insert_rows([header], 1)

# データをスプレッドシートに書き込む
worksheet.insert_rows(data_values, 2)  # 2行目からデータを挿入

{'spreadsheetId': '1-l-E6gb82lZQKlDSMpQX1LhH_z8bLG-u591WAzJv8p4',
 'updates': {'spreadsheetId': '1-l-E6gb82lZQKlDSMpQX1LhH_z8bLG-u591WAzJv8p4',
  'updatedRange': "'シート2'!A2:R694",
  'updatedRows': 693,
  'updatedColumns': 18,
  'updatedCells': 12474}}